## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(600)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=1c4fb111-ca5f-4911-8cdc-da6d05c3445a
Done
Done
ara-aragorn Done 280
Done
ara-arax Done 147
Done
ara-bte Done 94
Done
ara-unsecret Done 3
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2b_RHOBTB2_twohop.json
A.2b_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=58ccbc4d-9a8a-44c7-94cc-8c824c0aa238
Running
Done
ara-aragorn Done 0
Error
ara-arax Error 0
Error
ara-bte Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 

ara-unsecret Error 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Error
ara-explanatory Error 0
Done
ara-improving Done 10
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin.json
B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin
https://arax.ncats.io/?source=ARS&id=2f557037-1cf8-44c7-af33-ba4eccaeff61
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/

Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI.json
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
https://arax.ncats.io/?source=ARS&id=6328589d-0644-4567-a163-4f87d870dea7
Done
Done
ara-aragorn Done 0
Running
ara-arax Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease.json
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
https://arax.ncats.io/?source=ARS&id=7eec643b-a71f-458e-8b51-6556d12d9ca9
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2a_DILI-three-hop-from-MONDO:0005359_DILI.json
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
https://arax.ncats.io/?source=ARS&id=55eedab7-eab6-43cb-b2c2-b9d0be1d2b8a
Running
Running
ara-aragorn Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisord

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [5]:
dict_workflows

{'A.3_KCNMA1': '1c4fb111-ca5f-4911-8cdc-da6d05c3445a',
 'A.2b_RHOBTB2_twohop': '58ccbc4d-9a8a-44c7-94cc-8c824c0aa238',
 'A.2a_RHOBTB2_twohop': '7c31233e-30b1-46d3-8ae3-844a69c76d65',
 'A.8_EGFR_simple': 'be90d2d8-650d-48c1-b5ce-593b4d8a0ab8',
 'A.0_RHOBTB2_direct': 'dce800c5-92d0-448d-9ba8-24c6778b4b58',
 'A.2_RHOBTB2_twohop': 'ef2fb461-2015-4ee8-82f1-d02d4dfc0653',
 'A.9_EGFR_advanced': 'ceb10239-f062-4fbf-9951-330a18b369f5',
 'A.2b_expanded_RHOBTB2_twohop': '37eda025-90a0-4a86-9d26-7c6c11abbdcc',
 'A.1_RHOBTB2': '70f9a687-7424-40c4-9691-b3a675924bf0',
 'A.2a_expanded_RHOBTB2_twohop': 'ce77a3f7-b292-47b2-acd6-f42b0cd9fdb7',
 'B.5_DILI-one-hop-from-genes': 'f9f8bcd0-1379-4ce9-9d53-6f8aae920e88',
 'B.0_Initial_three-hop_MESH:D056487_ChronicDILI': '706d9f4d-bc98-441c-a2ad-dca2371a0587',
 'B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin': '881251fb-e144-4105-807d-827a8693afd6',
 'B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin': 'a520c8c8-0c78-4dd1-9c64-da6bf7ccca89',
 'B.3c_DIL

In [6]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 1c4fb111-ca5f-4911-8cdc-da6d05c3445a
Done
Done
ara-aragorn Done 280
Done
ara-arax Done 147
Done
ara-bte Done 94
Done
ara-unsecret Done 3
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
A.2b_RHOBTB2_twohop 58ccbc4d-9a8a-44c7-94cc-8c824c0aa238
Done
Done
ara-aragorn Done 0
Error
ara-arax Error 0
Error
ara-bte Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
A.2a_RHOBTB2_twohop 7c31233e-30b1-46d3-8ae3-844a69c76d65
Done
Done
ara-aragorn D

ara-aragorn Done 0
Error
ara-arax Error 0
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Error
kp-textmining Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Unknown
kp-chp Unknown 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease 07353b48-36e0-47ba-9c62-b3504eec63e6
Done
Done
ara-aragorn Done 0
Error
ara-arax Error 0
Error
ara-bte Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI b2ed5143-0c7e-4be8-aacc-e

ara-improving Done 3
Error
kp-cam Error 0
Error
kp-textmining Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Done
kp-cohd Done 64
Done
kp-icees-dili Done 0
D.6_metformin-ferritin c4f71660-6b22-4df0-8e3b-4d4019704081
Done
Done
ara-aragorn Done 116
Error
ara-arax Error 0
Error
ara-bte Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving Done 78
Error
kp-cam Error 0
Error
kp-textmining Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.2_ssri-heart-disease 483ad0c0-b702-4ce3-a44a-ae10239d186c
Done
Done
ara-aragorn Done 2028
Done
ara-arax Done 500
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving Done 32
Error
kp-cam Error 0
Error
kp-textmini

### Creating dataframe for workflows with PK

<br>

In [7]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2b_RHOBTB2_twohop
A.2b_expanded_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH:D00007738

In [8]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [9]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2b_RHOBTB2_twohop
A.2b_expanded_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH:D00007738

In [10]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:lincs',
               'infores:rtx-kg2',
               'infores:semmeddb',
               'infores:spoke'},
              'ara-arax': {'infores:arax',
               'infores:rtx-kg2',
               'infores:semmeddb'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:biothings-multiomics-biggim-drugresponse',
               'infores:dgi

In [11]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [12]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:spoke'],
 

In [13]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [14]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [15]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [16]:
df2test = pd.DataFrame(df2test.unstack().T)

In [17]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
icees:dili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn]
infores:aragorn-ranker-ara,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn]
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,NaN,NaN,[ara-bte],[ara-arax],[ara-arax],[ara-bte],NaN,[ara-bte],NaN,NaN,...,[ara-bte],NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-gtopdb,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,[ara-bte],NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hetio,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hmdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#df2test.drop([''], axis=0, inplace=True)

In [20]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [21]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
icees:dili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn]
infores:aragorn-ranker-ara,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn]
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],NaN
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,NaN,NaN,[ara-bte],[ara-arax],[ara-arax],[ara-bte],NaN,[ara-bte],NaN,NaN,...,[ara-bte],NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-gtopdb,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,[ara-bte],NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hetio,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hmdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#df2.replace([], 'None', regex=True,inplace=True)

In [23]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [24]:
df2test = df2test.rename_axis(None)

In [25]:
df2test.columns.name = None

In [26]:
df2test.fillna('', inplace=True)

In [27]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
icees:dili,,,,,,,,,[ara-arax],,...,,,,,,,,,,
infores:aragorn,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn]
infores:aragorn-ranker-ara,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn]
infores:arax,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],
infores:automat-biolink,,,,,,,,,,,...,[ara-bte],,,,,,,,,
infores:automat-ctd,,,[ara-bte],[ara-arax],[ara-arax],[ara-bte],,[ara-bte],,,...,[ara-bte],,[ara-arax],,,,,,,
infores:automat-gtopdb,,,[ara-bte],,,[ara-bte],,[ara-bte],,,...,,,[ara-arax],,,,,,,
infores:automat-hetio,,,[ara-bte],,,[ara-bte],,,,,...,,,[ara-arax],,,,,,,
infores:automat-hmdb,,,,,,,,,,,...,,,[ara-arax],,,,,,,
infores:automat-pharos,,,[ara-bte],,,[ara-bte],,[ara-bte],,,...,,,[ara-arax],,,,,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [28]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [29]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [30]:
infores_catalog = infores_catalog[:335]

In [31]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [32]:
dict_map

{'icees:dili': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'infores:biothings-multiomics-biggim-drugresponse': 'KP',
 'infores:biothings-multiomics-clinical-risk': 'KP',
 'infores:biothings-semmeddb-gene': 'KP',
 'infores:cam-kp': 'KP',
 'infores:chembl': 'External Source',
 'infores:civic': 'External Source',
 'infores:cohd': 'KP',
 'infores:connections-hypothesis': 'KP',
 'infores:ctd': 'External Source',
 'infores:dgidb': 'External Source',
 'infores:diseases': 'External Source',
 'infores:disgenet': 'External Source',
 'infores:drug-rep-hub': 'Illegal value',
 'infores:drugbank': 'External Source',
 'infores:drugc

In [33]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [34]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,...,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
icees:dili,,,,,,,,,[ara-arax],,...,,,,,,,,,,Illegal value
infores:aragorn,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],ARA
infores:aragorn-ranker-ara,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],[ara-aragorn],[ara-aragorn],,,,...,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],Illegal value
infores:arax,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,ARA
infores:automat-biolink,,,,,,,,,,,...,,,,,,,,,,KP
infores:automat-ctd,,,[ara-bte],[ara-arax],[ara-arax],[ara-bte],,[ara-bte],,,...,,[ara-arax],,,,,,,,KP
infores:automat-gtopdb,,,[ara-bte],,,[ara-bte],,[ara-bte],,,...,,[ara-arax],,,,,,,,KP
infores:automat-hetio,,,[ara-bte],,,[ara-bte],,,,,...,,[ara-arax],,,,,,,,KP
infores:automat-hmdb,,,,,,,,,,,...,,[ara-arax],,,,,,,,KP
infores:automat-pharos,,,[ara-bte],,,[ara-bte],,[ara-bte],,,...,,[ara-arax],,,,,,,,KP


In [35]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [36]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,...,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:biothings-multiomics-clinical-risk,,,,,,,,,,,...,,,[ara-bte],,,,,,KP,KP
infores:sri-ontology,,,,,,,,,,,...,,,[kp-cohd],,,[kp-cohd],,,KP,KP
infores:sri-reference-kg,,,,,,,,,,,...,,,,,,,,,KP,KP
infores:connections-hypothesis,,,,,,[kp-chp],,,,,...,[kp-chp],,,,,,,,KP,KP
infores:cohd,,,,,,,,,,,...,,,[kp-cohd],,,[kp-cohd],,,KP,KP
infores:lincs,[ara-improving],"[ara-aragorn, ara-improving]","[ara-bte, ara-improving]",[ara-aragorn],[ara-aragorn],[ara-improving],[ara-improving],[ara-improving],,,...,"[ara-arax, ara-improving]",,,,[ara-improving],,,,KP,KP
infores:mydisease-info,,,,,,,,,,,...,,,,,,,,,KP,KP
infores:cam-kp,,,,,,,,,,,...,,[ara-aragorn],,,,,,,KP,KP
infores:biothings-semmeddb-gene,,[ara-bte],[ara-bte],,,,,,,,...,,,,,,,,,KP,KP
infores:rtx-kg2,,"[ara-aragorn, ara-arax]",[ara-arax],[ara-arax],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-arax]",[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,"[ara-aragorn, ara-arax]","[ara-aragorn, ara-arax]",,[ara-arax],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-arax]",[ara-arax],[ara-aragorn],KP,KP


In [37]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [38]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [39]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [40]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [41]:
df.sort_index(inplace=True)

In [42]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,...,Results: 200,Results: 200,No Results: 422,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,No Results: 200,Results: 200
ara-arax,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 400,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,No Results: 200,Error: 500,Error: 598,No Results: 200,Results: 200,Error: 400,Results: 200,...,No Results: 200,Error: 598,No Results: 400,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-improving,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 400,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,No Results: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,No Results: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,Error: 598,No Results: 200,Error: 598,Error: 598,No Results: 200,Error: 598,Error: 502,Results: 200,Error: 598,Error: 598,...,Results: 200,No Results: 200,No Results: 502,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-cohd,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400,...,No Results: 200,Error: 400,No Results: 400,Results: 200,Error: 400,Error: 400,Error: 400,Results: 200,Error: 400,Error: 400


<br>

### Highlight the cells In Excel

<br>

In [43]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [44]:
styled = df.style.applymap(highlight)

In [45]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_three-hop_MESH:D056487_ChronicDILI,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI,B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.4a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 422,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,No Results: 200,Results: 200
ara-arax,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 400,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,No Results: 200,Error: 500,Error: 598,No Results: 200,Results: 200,Error: 400,Results: 200,Error: 598,Error: 500,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 400,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-improving,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 

In [46]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [47]:
wks_name

'Workflow Progress Tracker_2021_09_21-07_16_26_AM'

In [48]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [49]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2b_RHOBTB2_twohop,A.2b_expanded_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_three-hop_MESH:D056487_ChronicDILI,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI,B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.4a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 422,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,No Results: 200,Results: 200
ara-arax,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 400,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,No Results: 200,Error: 500,Error: 598,No Results: 200,Results: 200,Error: 400,Results: 200,Error: 598,Error: 500,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 400,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-improving,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 

In [50]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [51]:
#df.reset_index(inplace=True)

In [52]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

In [53]:
df.loc['pk'][1]

'=HYPERLINK("https://arax.ncats.io/?source=ARS&id=dce800c5-92d0-448d-9ba8-24c6778b4b58", "https://arax.ncats.io/?source=ARS&id=dce800c5-92d0-448d-9ba8-24c6778b4b58")'

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [54]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [55]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [56]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_21-07_16_26_AM' id:1423660451>

### Push Dataframe 2

<br>

In [57]:
wks2 = 'edge_attribute_source_' + date

In [58]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_21-07_16_26_AM' id:1579815368>

In [59]:
wks2

'edge_attribute_source_2021_09_21-07_16_26_AM'